In [1]:
from tempfile import tempdir
import SimpleITK as sitk
import numpy as np
import numpy.ma as ma
from matplotlib import pyplot
import matplotlib.pyplot as plt
import os
import sys
#import seaborn as sns
import UsefulFunctions as UF
import pandas as pd
import radiomics

In [10]:
image = "D:\\data\\prostateMR_radiomics\\nifti\\20fractions\\0000257\\MR36\\BaseImages\\0000257_MR36_image.nii"
mask = "D:\\data\\prostateMR_radiomics\\nifti\\20fractions\\0000257\\MR36\\Masks\\0000257_MR36_shrunk_pros.nii"

def MaskedMeanMed(image_url, mask_url):
    '''
    Returns mean and std of masked image
    Input - image path and mask path
    Output - mean and median (floats)
    '''

    image = sitk.ReadImage(image_url)
    image_array = sitk.GetArrayFromImage(image)
    
    mask = sitk.ReadImage(mask_url)
    mask_array = sitk.GetArrayFromImage(mask) 
    mask_array = mask_array / (np.max(mask_array))
    if image_array.shape != mask_array.shape:
        mean = np.nan
        med = np.nan
    
    else:
        masked_image = ma.array(image_array, mask=np.logical_not(mask_array), keep_mask=False, hard_mask=True)
        mean = ma.mean(masked_image)
        masked_image = masked_image[~masked_image.mask]
        med = ma.median(masked_image)
       
    return mean, med

mean, med = MaskedMeanMed(image, mask)
print(mean, med)

80.84616825932171 77.0


In [56]:
scaninfo = pd.read_csv("D:\\\\data\\Aaron\\ProstateMRL\\Data\\PatientKey.csv")
scaninfo.head()
patIDs = scaninfo.Patient.unique()
scaninfo.reset_index(drop=True)
#print(patIDs)
s, f = 0,0
res_df = pd.DataFrame()
treatments = scaninfo.Treatment.unique()
for t in treatments:
    scans = scaninfo.loc[scaninfo["Treatment"] == t] 
    treatmentIDs = scans.Patient.unique()
    print(len(treatmentIDs))
    for i in treatmentIDs:
        pat_df = scans.loc[scans["Patient"] == i]
        #print(pat_df.head())
        dates = pat_df.DateofScan.unique()
        sorted_df = pd.DataFrame()
        #print(i, dates)
        for j in dates:
            date_df = pat_df.loc[pat_df["DateofScan"] == j]
            #print(date_df.head(1))
            sorted_df = sorted_df.append(date_df.head(1))
        
        sorted_df["Scan"] = sorted_df["Scan"].str.replace(" MR", "")
        sorted_df["Scan"] = pd.to_numeric(sorted_df["Scan"])
        sorted_df = sorted_df.sort_values(["Scan"])
        sorted_df["Scan"] = "MR" + sorted_df["Scan"].astype(str)
        res_df = res_df.append(sorted_df)

print(res_df)
    

res_df.reset_index(drop=True, inplace=True)
res_df.to_csv("D:\\data\\Aaron\\ProstateMRL\\Data\\PatientKey_sorted.csv")

10
10
     Patient    Treatment  Scan  DateofScan
11       653         SABR   MR6    20111207
0        653         SABR  MR14    20111211
3        653         SABR  MR17    20111214
6        653         SABR  MR21    20111216
8        653         SABR  MR23    20111218
..       ...          ...   ...         ...
127   312108  20fractions  MR35    20191007
129   312108  20fractions  MR37    20191008
132   312108  20fractions  MR50    20191014
134   312108  20fractions  MR52    20191015
135   312108  20fractions  MR62    20191018

[98 rows x 4 columns]


In [69]:
mean_std = pd.read_csv("D:\data\Aaron\ProstateMRL\Data\MRLPacks\InterFractionChanges_v2\All_std.csv")

regions = mean_std.Region.unique()
norms = mean_std.Normalisation.unique()
patients = mean_std.PatID.unique()
print(patients)

for r in regions:
    region_df = mean_std.loc[mean_std.Region == r]

    norm_df = region_df.loc[region_df.Normalisation == "Raw"]
    stds = []
    means = []
    perc90 = []
    for p in patients:
        pat_df = norm_df.loc[norm_df.PatID == p]
        #print(r, p, np.mean(pat_df["StdSignal"]))
        std = np.mean(pat_df["StdSignal"])
        stds.append(std)
        mean = np.mean(pat_df["MeanSignal"])
        means.append(mean)
        percs = np.mean(pat_df["90thPerc"])
        perc90.append(percs)
    
    print(r, np.mean(stds), np.mean(means))#, np.mean(perc90))


            



[   213    231    257    292    626    695   1464 106958 251842 312108
    653    713    752    826   1088   1089   1118   1303   1307]
Prostate 32.44661970076236 61.848311894988704
Glute 24.06402131614344 12.809070375169222
Psoas 10.061523848040691 14.5320377283742
